In [1]:
import datetime as dt
from pathlib import Path

import pandas as pd

from pybenchmark.client import BenchmarkClient

In [2]:
URL = "https://benchmark-container-app.braveflower-04bf6ce2.uksouth.azurecontainerapps.io/"
DATA_DIR = Path("../resources")

In [3]:
client = BenchmarkClient(url=URL)

# Workouts

In [4]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="workouts")
df = df.rename(columns={"name": "routine_name"})
df.head()

,workoutroutine,date,exercise,weight,reps
0,Arm Block Chest,2024-10-28,Bench Press,72.5,8
1,Arm Block Chest,2024-10-28,Bench Press,72.5,5
2,Arm Block Chest,2024-10-28,Bench Press,60.0,11
3,Arm Block Chest,2024-10-28,Dips,0.0,10
4,Arm Block Chest,2024-10-28,Dips,0.0,10


In [5]:
latest_workout_date = client.workouts.get_latest_workout()["date"]
df = df[df["date"] > latest_workout_date]
df.head()

,workoutroutine,date,exercise,weight,reps
1175,Arm Block Back,2024-12-16,Pull Ups Neutral Grip,10.0,4
1176,Arm Block Back,2024-12-16,Pull Ups Neutral Grip,0.0,9
1177,Arm Block Back,2024-12-16,Dumbbell Prone Rows,35.0,7
1178,Arm Block Back,2024-12-16,Dumbbell Prone Rows,35.0,7
1179,Arm Block Back,2024-12-16,Dumbbell Prone Rows,30.0,12


In [6]:
dates = df["date"].unique()
dates = [str(date.date()) for date in dates]

In [7]:
exs = pd.DataFrame(client.exercises.get_exercises())
exs = exs.rename(columns={"id": "exercise_id", "name": "exercise_name"})
exs.head()

,exercise_name,exercise_id
0,bench press,1
1,lean-in cable row,2
2,dumbbell shoulder press neutral grip,3
3,lat pulldown neutral grip,4
4,rope hammer curls,5


In [8]:
routines = pd.DataFrame(client.workout_routines.get_workout_routines())
routines = routines.rename(columns={"name": "routine_name", "id": "routine_id"})
routines.head()

,routine_name,routine_id,routine_exercises,user_id
0,wave loading 321 lower 1,1,"[{'exercise': {'name': 'trap bar deadlift', 'i...",1
1,wave loading 321 upper 1,2,"[{'exercise': {'name': 'bench press', 'id': 1}...",1
2,wave loading 321 lower 2,3,"[{'exercise': {'name': 'barbell back squats', ...",1
3,wave loading 321 upper 2,4,"[{'exercise': {'name': 'military press', 'id':...",1
4,wave loading 642 lower 1,5,"[{'exercise': {'name': 'trap bar deadlift', 'i...",1


In [9]:
df["exercise_lower"] = df.exercise.str.lower()
df["workoutroutine_lower"] = df.workoutroutine.str.lower()
df.head()

,workoutroutine,date,exercise,weight,reps,exercise_lower,workoutroutine_lower
1175,Arm Block Back,2024-12-16,Pull Ups Neutral Grip,10.0,4,pull ups neutral grip,arm block back
1176,Arm Block Back,2024-12-16,Pull Ups Neutral Grip,0.0,9,pull ups neutral grip,arm block back
1177,Arm Block Back,2024-12-16,Dumbbell Prone Rows,35.0,7,dumbbell prone rows,arm block back
1178,Arm Block Back,2024-12-16,Dumbbell Prone Rows,35.0,7,dumbbell prone rows,arm block back
1179,Arm Block Back,2024-12-16,Dumbbell Prone Rows,30.0,12,dumbbell prone rows,arm block back


In [10]:
df_all = df.merge(right=exs, left_on="exercise_lower", right_on="exercise_name")
df_all = df_all.merge(
    right=routines, left_on="workoutroutine_lower", right_on="routine_name"
)[["date", "weight", "reps", "routine_id", "exercise_id"]]
df_all = df_all.set_index(["date", "routine_id", "exercise_id"])
df_all

weight  reps
date       routine_id exercise_id              
2024-12-16 13         15             10.0     4
                      15              0.0     9
                      17             35.0     7
                      17             35.0     7
                      17             30.0    12
                      38             45.0    10
                      38             45.0    10
                      38             35.0    15
                      48             45.0    10
                      48             45.0    10
                      48             35.0    15
                      33             10.0    20
                      33             10.0    15
                      33             10.0    13
                      40             24.0    60
                      40             24.0    50
2024-12-17 12         1              72.5     8
                      1              72.5     8
                      1              62.5    12
                      36             15.0    13
                      36             15.0    13
                      36             15.0    11
                      30             70.0    10
                      30             55.0    15
                      6               7.5    10
                      6               5.0    18
                      6               5.0    18
                      8              20.0    20
                      8              20.0    20
                      8              20.0    15
                      39              0.0    20
                      39              0.0    20
2024-12-18 15         9             115.0     8
                      9              95.0    15
                      41            127.5     8
                      41            102.5    15
                      22             30.0    10
                      22             30.0    10
                      22             30.0    10
                      42             50.0    15
                      42             50.0    15
                      42             50.0    15
                      43             35.0    20
                      43             35.0    20
                      43             35.0    20

In [11]:
for date in dates:
    all_workouts = []
    routines = df_all.loc[date].index.get_level_values(0).unique()
    for routine in routines:
        all_exercises = []
        exercises = df_all.loc[(date, routine)].index.unique()
        for exercise in exercises:
            sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
            data = {"id": exercise, "sets": sets}
            all_exercises.append(data)
        try:
            response = client.workouts.create_workout(
                date=str(date),
                exercises=all_exercises,
                workoutroutine_id=routine,
                user_id=1,
            )
            print(f"Created Workout for date {date}")
        except Exception as e:
            print(e)

C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.lo

Created Workout for date 2024-12-16


C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.lo

Created Workout for date 2024-12-17


C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8200\3749088168.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.lo

Created Workout for date 2024-12-18


# Weights

In [18]:
last_weight = client.weights.get_weights(user_id=1)[0]
last_weight_date = dt.datetime.strptime(last_weight["date"], "%Y-%m-%d").date()
last_weight_date

datetime.date(2024, 12, 13)

In [13]:
WEEKDAYS = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3,
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6,
}

In [14]:
df = pd.read_excel(DATA_DIR / "weights.xlsx", sheet_name="Sheet2")
df = df.melt(
    id_vars="Date",
    value_vars=[
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday",
    ],
)
df["dow"] = df["variable"].map(WEEKDAYS)
df["Real Date"] = df.apply(axis=1, func=lambda x: dt.timedelta(days=x.dow) + x.Date)
df = (
    df.drop(columns=["Date", "dow", "variable"])
    .rename(columns={"value": "weight", "Real Date": "date"})
    .dropna()
)
weights = df.to_dict(orient="records")
weights

[{'weight': 60.0, 'date': Timestamp('2022-02-07 00:00:00')},
 {'weight': 60.9, 'date': Timestamp('2022-02-14 00:00:00')},
 {'weight': 60.0, 'date': Timestamp('2022-02-28 00:00:00')},
 {'weight': 60.6, 'date': Timestamp('2022-03-14 00:00:00')},
 {'weight': 60.8, 'date': Timestamp('2022-03-21 00:00:00')},
 {'weight': 61.9, 'date': Timestamp('2022-03-28 00:00:00')},
 {'weight': 62.1, 'date': Timestamp('2022-04-25 00:00:00')},
 {'weight': 62.2, 'date': Timestamp('2022-05-30 00:00:00')},
 {'weight': 63.5, 'date': Timestamp('2022-06-13 00:00:00')},
 {'weight': 64.6, 'date': Timestamp('2022-06-20 00:00:00')},
 {'weight': 64.0, 'date': Timestamp('2022-07-04 00:00:00')},
 {'weight': 64.2, 'date': Timestamp('2022-07-11 00:00:00')},
 {'weight': 65.0, 'date': Timestamp('2022-08-22 00:00:00')},
 {'weight': 64.0, 'date': Timestamp('2022-08-29 00:00:00')},
 {'weight': 63.1, 'date': Timestamp('2022-09-05 00:00:00')},
 {'weight': 61.8, 'date': Timestamp('2022-09-12 00:00:00')},
 {'weight': 61.6, 'date'

In [15]:
weights = [weight for weight in weights if weight["date"].date() > last_weight_date]
weights

[{'weight': 64.3, 'date': Timestamp('2024-12-09 00:00:00')},
 {'weight': 63.7, 'date': Timestamp('2024-12-10 00:00:00')},
 {'weight': 64, 'date': Timestamp('2024-12-11 00:00:00')},
 {'weight': 64.4, 'date': Timestamp('2024-12-13 00:00:00')}]

In [16]:
for weight in weights:
    value = weight["weight"]
    date = weight["date"].date()
    try:
        response = client.weights.create_weight(weight=value, date=str(date), user_id=1)
        print(f"Created Weight {value}")
    except Exception as e:
        print(e)

Created Weight 64.3
Created Weight 63.7
Created Weight 64
Created Weight 64.4
